<a href="https://colab.research.google.com/github/aisha-partha/AIMLOps-MiniProjects/blob/mp_4/M2_NB_MiniProject_2_DeepLabV3%2B_Lungs_Segmentation_partA_Shivayogi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Programme in AI and MLOps
## A programme by IISc and TalentSprint
### Mini-Project: Lung Segmentation of Chest X-Ray dataset using DeepLabV3+

## Learning Objectives:

At the end of the experiment, you will be able to:

- understand, prepare, and visualize the the dataset containing image and corresponding masked image used for segmentation
- implement DeepLabV3+ architecture
- create a masked image (prediction)

## Introduction

Semantic segmentation is a computer vision task that involves dividing an image into different regions, each of which is labeled with a semantic category. The goal of semantic segmentation is to enable machines to understand the content of an image at a pixel level, by assigning a label to each individual pixel based on the object or region it belongs to.

This technique is widely used in many applications such as self-driving cars, medical image analysis, and object recognition in robotics. It helps to extract meaningful information from images and to understand the relationships between objects and their environment.

The below figure shows how semantic segmentation differs from other algorithms, such as object detection.

<br>
<img src='https://cdn.iisc.talentsprint.com/AIandMLOps/Images/segmentation_vs_other_algos.jpeg' width=700px>
<br><br>

Moreover, in contrast to object detection, which detects and localizes objects within an image, semantic segmentation is more precise and detailed. It provides a much more granular understanding of the content of an image, allowing for more advanced and accurate applications.

## Dataset

*  The Chest X-Ray dataset is made up of images and segmentated mask from two diffrent sources - Shenzhen and Montgomery dataset
*  The CXR_png folder consists of Chest X-Rays and the masks folder has the segmented mask
*  There are 704 images with their masks mapped with each other  

<br>
$\quad$<img src='https://cdn.extras.talentsprint.com/AIandMLOps/Images/semantic_segmentation.png' width=700px>
<br><br>


## Problem Statement

Perfrom the lungs segmentation on Chest X-Ray dataset using DeepLabV3+ model.

## Grading = 10 Points

In [1]:
#@title Download the data
!wget https://cdn.iisc.talentsprint.com/AIandMLOps/MiniProjects/Datasets/LungSegmentation.zip
!unzip -qq LungSegmentation.zip
print("Data Downloaded Successfuly!!")

--2024-05-30 09:38:11--  https://cdn.iisc.talentsprint.com/AIandMLOps/MiniProjects/Datasets/LungSegmentation.zip
Resolving cdn.iisc.talentsprint.com (cdn.iisc.talentsprint.com)... 172.105.52.210
Connecting to cdn.iisc.talentsprint.com (cdn.iisc.talentsprint.com)|172.105.52.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 139904834 (133M) [application/zip]
Saving to: ‘LungSegmentation.zip’

LungSegmentation.zi 100%[===================>] 133.42M  10.8MB/s    in 16s     

2024-05-30 09:38:28 (8.42 MB/s) - ‘LungSegmentation.zip’ saved [139904834/139904834]

Data Downloaded Successfuly!!


### Import required packages

In [2]:
import os
import cv2
import numpy as np
from glob import glob
from PIL import Image
from scipy.io import loadmat
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### **PART-A**

### Data Visualization (1 point)

#### Image Visualization
Visualize one of the Chest X-Ray image and its segmented label image.

In [ ]:
# Visualize an image
# YOUR CODE HERE

In [ ]:
# Visualize a semantic part segmentation label image
# YOUR CODE HERE

### Create a TensorFlow Dataset (1 points)
Use the image size, batch size, number of classes, and data directory as given in the below code cell.

Hint:
1. Find all the image files in the CXR_png and masks subdirectory separately. The resulting list should be sorted in ascending order.
2. Create a list of validation image, validation masks, test image, test masks, etc.
3. Create a function that reads an image file and returns a preprocessed image tensor.  

In [13]:
IMAGE_SIZE = 512
BATCH_SIZE = 4
NUM_CLASSES = 2
DATA_DIR = "./LungSegmentation"

# Find paths for all the image files in the 'CXR_png' subdirectory of the DATA_DIR directory.
# Sort the list of file paths in ascending order
#all_images = # YOUR CODE HERE
import os
import re

def find_numbered_files(directory):
    pattern = re.compile(r'\d+')
    numbered_files = [f for f in os.listdir(directory) if pattern.search(f)]
    return numbered_files

# Replace '/path/to/folder' with the actual path to your directory
#directory_path = '/path/to/folder'
all_images = sorted(find_numbered_files(DATA_DIR + "/CXR_png"))

#print("Numbered files in the folder:")
#for file in all_images:
#    print(file)


# Find paths for all the mask files in the 'masks' subdirectory of the DATA_DIR directory.
# Sort the list of file paths in ascending order
all_masks = sorted(find_numbered_files(DATA_DIR + "/masks"))
#print("Numbered files in the folder:")
#for file in all_masks:
#    print(file)


# Create a list of validation image files
val_images = []
for file in all_images[301:400]:
    val_images.append(file)
print(val_images)



# Create a list of validation mask files
val_masks = []
for file in all_masks[301:400]:
    val_masks.append(file)
print(val_masks)

# Create a list of test image files
test_images = []
for file in all_images[201:300]:
    test_images.append(file)

# Create a list of test mask files
test_masks = []
for file in all_masks[201:300]:
    test_masks.append(file)


# Create a list of train image files
train_images = []
for file in all_images[:200]:
    train_images.append(file)

# Create a list of train mask files
train_masks = []
for file in all_masks[:200]:
    train_masks.append(file)


['CHNCXR_0370_1.png', 'CHNCXR_0371_1.png', 'CHNCXR_0372_1.png', 'CHNCXR_0373_1.png', 'CHNCXR_0374_1.png', 'CHNCXR_0375_1.png', 'CHNCXR_0376_1.png', 'CHNCXR_0377_1.png', 'CHNCXR_0378_1.png', 'CHNCXR_0379_1.png', 'CHNCXR_0380_1.png', 'CHNCXR_0381_1.png', 'CHNCXR_0382_1.png', 'CHNCXR_0383_1.png', 'CHNCXR_0384_1.png', 'CHNCXR_0385_1.png', 'CHNCXR_0386_1.png', 'CHNCXR_0387_1.png', 'CHNCXR_0388_1.png', 'CHNCXR_0389_1.png', 'CHNCXR_0390_1.png', 'CHNCXR_0391_1.png', 'CHNCXR_0392_1.png', 'CHNCXR_0393_1.png', 'CHNCXR_0394_1.png', 'CHNCXR_0395_1.png', 'CHNCXR_0396_1.png', 'CHNCXR_0397_1.png', 'CHNCXR_0398_1.png', 'CHNCXR_0399_1.png', 'CHNCXR_0400_1.png', 'CHNCXR_0401_1.png', 'CHNCXR_0402_1.png', 'CHNCXR_0403_1.png', 'CHNCXR_0404_1.png', 'CHNCXR_0405_1.png', 'CHNCXR_0406_1.png', 'CHNCXR_0407_1.png', 'CHNCXR_0408_1.png', 'CHNCXR_0409_1.png', 'CHNCXR_0410_1.png', 'CHNCXR_0411_1.png', 'CHNCXR_0412_1.png', 'CHNCXR_0413_1.png', 'CHNCXR_0414_1.png', 'CHNCXR_0415_1.png', 'CHNCXR_0416_1.png', 'CHNCXR_0417

In [4]:
# Function to read an image file and returns a preprocessed image tensor.
# The mask argument is set to False by default, indicating that it is an image file, not a mask file.

import tensorflow as tf
from PIL import Image
import numpy as np

'''
def read_image(image_path, mask=False):
    """
    Load an image file and preprocess it into a tensor.

    Args:
    image_path (str): Path to the image file.
    target_size (tuple): The target size of the image after resizing (height, width).

    Returns:
    tf.Tensor: Preprocessed image tensor.
    """
    # Load the image file
    image = Image.open(image_path)
    # Resize the image
    image = image.resize((224,224)), Image.ANTIALIAS)
    # Convert the image to a numpy array and normalize the pixel values
    image_array = np.array(image) / 255.0
    # Convert the numpy array to a TensorFlow tensor
    image_tensor = tf.convert_to_tensor(image_array, dtype=tf.float32)
    # Add a batch dimension
    image_tensor = tf.expand_dims(image_tensor, axis=0)
    return image_tensor

# Example usage
image_tensor = read_image(DATA_DIR + "/CXR_png" + "/CHNCXR_0010_0.png" )
print(image_tensor.shape)

'''

def read_image(file_path):
    # Reads an image from a file, decodes it into a dense tensor, and resizes it
    # to a fixed shape.
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=3)  # Adjust channels according to your image format
    image = tf.image.resize(image, [256, 256])  # Resize images if required
    return image



Create a **load_data** function that takes in two arguments, **image_list** and **mask_list**, which are lists of file paths to the images and corresponding masks, respectively. It then reads in the image and mask using the **read_image** function defined earlier. The function returns a tuple of **image** and **mask**.

In [5]:
def load_data(image_path, mask_path):
    # Load the image and mask and prepare them for processing.
    image = read_image(image_path)
    mask = read_image(mask_path)
    return image, mask

Create a **data_generator** function that takes in **image_list** and **mask_list** as arguments.

The function should:
- create a **tf.data.Dataset** object from the input data
- map the **load_data** function to each element in the dataset
- convert the dataset into batches of size **BATCH_SIZE**, drop any incomplete batch at the end of the dataset
- return the resulting dataset

In [29]:
# Generate train and validation set

def data_generator(image_list, mask_list, BATCH_SIZE):
    # Create a tf.data.Dataset from image and mask paths.
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    # Apply the load_data function to each item in the dataset.
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    # Batch the data and drop any incomplete batches.
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    # Prefetch data for enhanced performance.
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Example usage
train_image_list = [DATA_DIR  + "/CXR_png" + "/CHNCXR_0010_0.png", DATA_DIR + "/CXR_png" + "/CHNCXR_0011_0.png"]  # Add your actual image paths here
train_mask_list = [DATA_DIR + "/masks" + "/CHNCXR_0010_0.png" , DATA_DIR + "/masks" + "/CHNCXR_0011_0.png"]  # Add your actual mask paths here
BATCH_SIZE = 2  # Set your batch size as needed

dataset = data_generator(train_image_list, train_mask_list, BATCH_SIZE)
for images, masks in dataset.take(1):  # Process one batch
    print("train_dataset_img = " +  str(images.shape))
    print("train_dataset_msk = " + str(masks.shape))  # Print shapes to verify
batch_count_train = sum(1 for _ in dataset)
print(batch_count_train)

#batch_count = dataset.cardinality().numpy()
#print("Total number of batches:", batch_count_train)


val_img_list = [DATA_DIR  + "/CXR_png/" + val_images[5], DATA_DIR  + "/CXR_png/" + val_images[6]]
val_msk_list = [DATA_DIR  + "/masks/" + val_masks[5], DATA_DIR  + "/masks/" + val_masks[6]]

dataset = data_generator(val_img_list, val_msk_list, BATCH_SIZE)
for images, masks in dataset.take(1):  # Process one batch
    print("val_dataset_img = " + str(images.shape))
    print("val_dataset_msk = " + str(masks.shape))  # Print shapes to verify

batch_count_val = sum(1 for _ in dataset)
print(batch_count_val)

#train_dataset = data_generator(# YOUR CODE HERE)
#val_dataset = data_generator(# YOUR CODE HERE)

train_dataset_img = (2, 256, 256, 3)
train_dataset_msk = (2, 256, 256, 3)
1
val_dataset_img = (2, 256, 256, 3)
val_dataset_msk = (2, 256, 256, 3)
1


Check the total images for train and validation.

In [30]:
# YOUR CODE HERE
imgs_train = 2
imgs_val = 2
total_images = imgs_train + imgs_val


Check the number of batches formed in train and validation set.


In [31]:
# YOUR CODE HERE
total_batches = batch_count_train + batch_count_val
print(total_batches)

2


Now, we have created our TensorFlow Dataset. Further, let us try to understand the DeepLabV3+ model.

### **PART-B**

### Building the DeepLabV3+ model

Downsampling is widely adopted in deep convolutional neural networks (CNN) for reducing memory consumption while preserving the transformation invariance to some degree.

Multiple downsampling of a CNN will lead the feature map resolution to become smaller, resulting in lower prediction accuracy and loss of boundary information in semantic segmentation.

DeepLabv3+ helps in solving these issues by including **atrous convolutions**. They aggregate context around a feature which helps in segmenting it better.

<br>

#### **Atrous Convolution/Dilated Convolution**

It is a tool for refining the effective field of view of the convolution. It modifies the field of view using a parameter termed ***atrous rate*** or ***dilation rate***.

With dilated convolution, as we go deeper in the network, we can keep the stride constant but with larger field-of-view without increasing the number of parameters or the amount of computation. It also enables larger output feature maps, which is useful for semantic segmentation.

In the below figure, Atrous/Dilated Convolution has wider field of view with same number of parameters as Normal convolution.

<br>
<img src="https://cdn.iisc.talentsprint.com/AIandMLOps/Images/Dilated_Conv.jpg" width=500px>
<br><br>



#### **DeepLabv3+**

Earlier version, DeepLabv3 has a problem of consuming too much time to process high-resolution images. DeepLabv3+ is a semantic segmentation architecture that improves upon DeepLabv3 with several improvements, such as adding an effective decoder module to refine the segmentation results.

The below figure shows the typical architecture of DeepLabv3+. The encoder module processes multiscale contextual information by applying dilated/atrous convolution at multiple scales, while the decoder module refines the segmentation results along object boundaries.

<br>
<img src="https://cdn.iisc.talentsprint.com/AIandMLOps/Images/deeplabv3_plus_model.png" width=1000px>
<br><br>

Deeplabv3+ employs Aligned Xception network as its main feature extractor (encoder), although with substantial modifications. Depth-wise separable convolution replaces all max pooling procedures.

In Model Playground, we can select feature extraction (encoding) network to use as either **Resnet** or EfficientNet.

The reason for using **Dilated Spatial Pyramid Pooling** is that it was shown that as the sampling rate becomes larger, the number of valid filter weights (i.e., weights that are applied to the valid feature region, instead of padded zeros) becomes smaller.


### Create the model (5 points)
First, create different functions  to implement DeepLabV3+ architecture.



Create a function, **convolution_block()**, to add a convolution layer, a BatchNormalization layer, and apply ReLu activation in one go.

In [ ]:
# YOUR CODE HERE

Create another function to perform **Dilated Spatial Pyramid Pooling**. Use above function to add different convolution blocks.

In [ ]:
# YOUR CODE HERE

The encoder features are first bilinearly upsampled by a factor 4, and then concatenated with the corresponding low-level features from the network backbone that have the same spatial resolution. Here, we use a **ResNet50** pretrained on ImageNet as the backbone model, and we use
the low-level features from the `conv4_block6_2_relu` block of the backbone.

##### Exploring ResNet-50 architecture before using it.

In [ ]:
res_input = keras.Input(shape=(128, 128, 3))
resnet50 = keras.applications.ResNet50(weights="imagenet", include_top=False, input_tensor = res_input)

# Layers present in ResNet-50 network
resnet50.summary()

From the above layers,

- Use the low-level features from the `conv2_block3_2_relu` layer of the ResNet-50 network to fead in Decoder.

- Use the features from the `conv4_block6_2_relu` layer of the ResNet-50 to fead in Dilated Spatial Pyramid Pooling module.

#### Create Encoder
Create a function to implement the architecture for Encoder block. Use **ResNet50** pretrained on ImageNet as the backbone network. Use the features from the **conv4_block6_2_relu** layer of the backbone to fead in Dilated Spatial Pyramid Pooling module. Then return the backbone network along with encoder output.

In [ ]:
# YOUR CODE HERE

#### Create Decoder

Create a function to implement the architecture for Decoder block. The encoder features are first bilinearly upsampled by a factor 4, and then concatenated with the corresponding low-level features (the **conv2_block3_2_relu** layer) from the network backbone that have the same spatial resolution.

In [ ]:
# YOUR CODE HERE

#### Building full architecture of the Model

Write a function that combines the encoder and decoder functions defined above to create & implement a complete  DeepLabV3+ architecture.

In [ ]:
# YOUR CODE HERE

In [ ]:
# Create model

### Training (1 point)

We train the model using sparse categorical crossentropy as the loss function, and
Adam as the optimizer.

##### Compile model


In [ ]:
# YOUR CODE HERE

##### Model Training

In [ ]:
# YOUR CODE HERE

We can plot the training and validation loss to see how the training went. This should show generally decreasing values per epoch.

#####  Learning curve from model history (1 point)

In [ ]:
# Function to plot learning curves
# YOUR CODE HERE

In [ ]:
# Display learning curves
# YOUR CODE HERE

### Visualize Predictions (1 points)

In [ ]:
# Inference from model
# YOUR CODE HERE

#### Plot the predictions

In [ ]:
# YOUR CODE HERE